# OPC2MQTT  
OPC2MQTT是从OPC获取数据以后通过MQTT发送,MQTT2OPC是接收MQTT的数据写入OPC,相应的各自的CSV配置为opc2mqtt.csv和mqtt2opc文件。

In [1]:
# app.py

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# create a file handler
handler = logging.FileHandler('opc2mqtt.log',encoding='utf-8')
handler.setLevel(logging.INFO)

# create a logging format
formatter = logging.Formatter('%(asctime)s,%(name)s,%(levelname)s,%(message)s')
handler.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(handler)

### STEP1: 连接OPC服务器

In [2]:
""" Step 1 """
from opcua import Client as OPCUAClient

In [3]:
from opcua import ua

In [4]:
uri = "opc.tcp://192.168.10.1:4840"

In [5]:
""" Step 2 """
client = OPCUAClient(uri)

In [6]:
try:
    client.connect() # connect to the data source
except Exception as e:
    logging.exception(e)

Requested session timeout to be 3600000ms, got 30000ms instead


### STEP2:连接MQTT服务器

In [7]:
import paho.mqtt.client as mqtt #import the client1
broker_address="broker.emqx.io"
#broker_address="iot.eclipse.org" #use external broker
mqttc = mqtt.Client() #create new instance
mqttc.connect(broker_address) #connect to broker

0

### STEP3: 加载CSV配置文件

https://thispointer.com/python-read-csv-into-a-list-of-lists-or-tuples-or-dictionaries-import-csv-to-list/

In [8]:
import csv

In [9]:
with open('opc2mqtt.csv', newline='') as f:
    reader = csv.reader(f)
    header = next(reader)
    list_reader = list(reader)
    print(list_reader)

[['srm/opc2mqtt/INT_NUM', 'ns=3;s="OPC2MQTT"."INT_NUM"']]


In [10]:
list_topic = [i[0] for i in list_reader]

In [11]:
list_topic

['srm/opc2mqtt/INT_NUM']

In [12]:
list_nodeid = [i[1] for i in list_reader]

In [13]:
list_nodeid

['ns=3;s="OPC2MQTT"."INT_NUM"']

### STEP4: 读取OPC中的数据通过MQTT发送

In [14]:
import json

In [15]:
def read_opc(list_topic, list_nodeid):
    for i in range(len(list_topic)):
        value = client.get_node(list_nodeid[i]).get_value()
        display = client.get_node(list_nodeid[i]).get_display_name().Text
        mqttc.publish(list_topic[i],json.dumps(value))
        
        logger.info(f'{display},{json.dumps(value)}')

In [ ]:
import time

mqttc.loop_start()
    
while True:
    time.sleep(3)  
    read_opc(list_topic, list_nodeid)